In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import sys
import os


from sklearn.metrics import r2_score
sys.path.append('../')
sys.path.append('../../')


In [2]:
from Bio.SeqIO import read, parse
import orthoani

In [3]:
from db_sketching.kmer_set import KMerSet, FracMinHash

In [4]:
def cond(kmer_hash):
    hash = (976369 * kmer_hash + 1982627) % 10000
    if hash < 50:
        return True
    else:
        return False
    
random.seed(10)

mod = 1000000007
r0 = random.randint(0,1000000006)
r1 = random.randint(0,1000000006)
r2 = random.randint(0,1000000006)
r3 = random.randint(0,1000000006)
c = 200

def poly_cond(kmer_hash):
    hash_mod = kmer_hash % mod
    hash = (r0 + hash_mod * (r1 + hash_mod * (r2 + hash_mod * (r3)))) % mod
    return hash < (mod / c)

In [5]:
def compare_ANI(genome_file_1, genome_file_2, kmer_length, kmer_class, sketching_condition):
    """
    Function to compute the estimated ANI using FracMinHash 
    and compare it to the value from OrthoANI
    """
    return compute_ortho_ani(genome_file_1, genome_file_2), compute_kmer_ani(genome_file_1, genome_file_2, kmer_length, kmer_class, sketching_condition)

def compute_kmer_ani(genome_file_1, genome_file_2, kmer_length, kmer_class, sketching_condition):
    genome_1_kmer = kmer_class(sketching_condition, kmer_length)
    genome_1_kmer.insert_file(genome_file_1)

    genome_2_kmer = kmer_class(sketching_condition, kmer_length)
    genome_2_kmer.insert_file(genome_file_2)
    
    kmer_estimated_ani = genome_1_kmer.ANI_estimation(genome_2_kmer)

    return kmer_estimated_ani

def compute_ortho_ani(genome_file_1, genome_file_2):
    genome_1_read = parse(genome_file_1,"fasta")
    genome_2_read = parse(genome_file_2,"fasta")
    ortho_ani_value = orthoani.orthoani(genome_1_read,genome_2_read)

    return ortho_ani_value




In [6]:
data_dir = "../../data_temp/"
genome_files = []

for collection in os.listdir(data_dir):
    collection_dir = os.path.join(data_dir,collection)
    for filename in os.listdir(collection_dir):
        full_filename = os.path.join(collection_dir,filename)
        genome_files.append(full_filename)

print(genome_files)


['../../data_temp/388449/7.fna', '../../data_temp/388449/2.fna', '../../data_temp/388449/9.fna', '../../data_temp/388449/5.fna', '../../data_temp/388449/4.fna', '../../data_temp/388449/1.fna', '../../data_temp/388449/8.fna', '../../data_temp/388449/3.fna', '../../data_temp/388449/0.fna', '../../data_temp/388449/6.fna', '../../data_temp/1902584/1.fna', '../../data_temp/1902584/0.fna', '../../data_temp/1331736/7.fna', '../../data_temp/1331736/2.fna', '../../data_temp/1331736/9.fna', '../../data_temp/1331736/5.fna', '../../data_temp/1331736/4.fna', '../../data_temp/1331736/1.fna', '../../data_temp/1331736/8.fna', '../../data_temp/1331736/3.fna', '../../data_temp/1331736/0.fna', '../../data_temp/1331736/6.fna', '../../data_temp/1892252/7.fna', '../../data_temp/1892252/2.fna', '../../data_temp/1892252/9.fna', '../../data_temp/1892252/5.fna', '../../data_temp/1892252/4.fna', '../../data_temp/1892252/1.fna', '../../data_temp/1892252/8.fna', '../../data_temp/1892252/3.fna', '../../data_temp/18

In [18]:
from multiprocess import Pool

max_pool = 16

In [8]:
lim_file = 100

In [9]:
ortho_vals = [] 

def compute_ortho_ani_parallel(genome_files):
    args = [(g1,g2) for g1,g2 in zip(genome_files[:lim_file],genome_files[1:lim_file])]
    with Pool(max_pool) as p:
        return p.starmap(compute_ortho_ani,args)
    
ortho_vals = compute_ortho_ani_parallel(genome_files)

In [10]:
kmer_length = 21
kmer_class = FracMinHash

def compute_kmer_ani_parallel(genome_files, kmer_length):
    args = [(g1,g2,kmer_length,kmer_class,poly_cond) for g1,g2 in zip(genome_files[:lim_file],genome_files[1:lim_file])]
    with Pool(max_pool) as p:
        return p.starmap(compute_kmer_ani,args)
    


In [12]:
print(ortho_vals)

[0.6365614184397164, 0.6844516210045662, 0.877343023308958, 0.6291772486772487, 0.629662520661157, 0.6879192810150376, 0.689969785670545, 0.6883388260869565, 0.7031486872727273, 0.5998172549019608, 0.7113680276816609, 0.6274515384615384, 0.7458501844941957, 0.8596176090116279, 0.746394787008708, 0.7449114282330649, 0.7452413389021479, 0.8185315478026214, 0.7445563756368689, 0.8153205903145988, 0.7458778778937812, 0.6237357333333333, 0.6924438438914027, 0.9074765761743169, 0.6796802946768061, 0.6796657196087564, 0.9767430027998133, 0.6767753015616585, 0.6727879976442874, 0.6999582350292661, 0.6815503967065868, 0.6243225471698113, 0.7410200523731587, 0.7275068041634541, 0.6121577419354839, 0.7543466666666667, 0.749216392733564, 0.7252024251805985, 0.6260803465346535, 0.8673194349206349, 0.9840955604938272, 0.877905855774985, 0.7003834817813765, 0.6992046968403074, 0.7218278381458967, 0.727200487197724, 0.7024489756507137, 0.6972845484477893, 0.5920594736842105, 0.7647226716016151, 0.7515

In [26]:
kmer_vals = [[]] * 40
for kmer_length in range(20,29+1):
    kmer_vals[kmer_length] = compute_kmer_ani_parallel(genome_files, kmer_length)

print(kmer_vals)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [0.6453364737021984, 0.6912975684982525, 0.8963348888236039, 0.6069124388565897, 0.0, 0.7275494224804759, 0.6687456553483044, 0.7479561136239409, 0.7535558631604059, 0.0, 0.7173192885277893, 0.0, 0.800924801140114, 0.8922433844299519, 0.8013634900672709, 0.8065734166853711, 0.7927693424470061, 0.8590327490353102, 0.8014098723021393, 0.8492847858390624, 0.8094886503034265, 0.6323586755500133, 0.7285154979046089, 0.9421448569478974, 0.7168199488769366, 0.7242718849250752, 0.9890867591396623, 0.693389744403138, 0.6868989883469758, 0.7323163840719438, 0.7338398115930926, 0.0, 0.7919564033678363, 0.7809866709137117, 0.0, 0.8203177227403905, 0.819067300338463, 0.796464061539167, 0.0, 0.878907937206215, 0.9835736081429343, 0.8843674972083813, 0.7438406865146765, 0.7152082738016154, 0.7631914550833789, 0.7760882277070803, 0.7388505766852751, 0.7308045363213244, 0.6255090582825225, 0.8110302844703009, 0.80599909692

In [41]:
fig, ax = plt.subplots(nrows=10,figsize=(6,60),dpi=600)

for kmer_length in range(20,29+1):
    filter_o, filter_k = zip(*[(o,k) for o,k in zip(ortho_vals,kmer_vals[kmer_length]) if k > 0])
    lin_fit = np.poly1d(np.polyfit(filter_o, filter_k, 1))
    ax[kmer_length-20].plot(ortho_vals,ortho_vals,"r-")
    ax[kmer_length-20].scatter(filter_o,filter_k,marker="^",s=[2 for _ in filter_o])
    ax[kmer_length-20].plot(np.unique(ortho_vals), lin_fit(np.unique(ortho_vals)))
    print(lin_fit)

    ax[kmer_length-20].legend(["OrthoANI",f"{kmer_length}-mer dots",f"{kmer_length}-mer line : {lin_fit}"])


plt.plot()


 
0.9592 x + 0.06758
 
0.9571 x + 0.06999
 
0.9479 x + 0.07954
 
0.9031 x + 0.1184
 
0.8748 x + 0.1429
 
0.8275 x + 0.1837
 
0.8302 x + 0.183
 
0.8076 x + 0.2025
 
0.775 x + 0.2321
 
0.7513 x + 0.2529


[]

<Figure size 640x480 with 0 Axes>

In [ ]:
print(ortho_vals)

[]
